<a href="https://colab.research.google.com/github/Juntolli/comp215/blob/main/labs/lab03-classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 3
----------------
#### Name(s): Bruno Juntolli
#### Date: January 23, 2025

By the end of this lab you should be able to:
  * use a `class` to organize related data and functions
  * identify a `record` as a related collection of data, with fields for each data value

During this lab, you will be introduced to the following:
  * the `@dataclass` decorator to simplify class definition for classes that primarily encapsulate a data structure.
  * the `@classmethod` decorator to create 'object factories'.
  * plotting frequency distributions in `matplotlib`



## Near Earth Objects (with classes)

We'll continue working with Near Earth Object data using NASA's API: https://api.nasa.gov/.  In this lab we will define an Asteroid class to help us structure the data for easier data analysis.  Here's a brief review from Lab 2 on how to use the API.  

In [2]:
# import necessary modules
import datetime, json, requests
from dataclasses import dataclass
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline

### Review: make an API query

Here's a query that gets the record for a single NEO that recently passed by.  Read these functions carefully to be sure you know what they are returning.  Note the nested for loop structure in the return statement of the `get_neos` function - be sure to understand what it is doing.  You might want to add some more `pprint` statements to be sure you understand the data structures.

In [7]:
API_KEY = 'BI8itZMCmoAG5dg7eFf0yzdNDiXysbuxj9ig1qYM'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    print(f'{data["element_count"]} Near Earth Objects found for week of {start_date}')
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

week_start = '2023-01-15'
neos = get_neos(week_start)
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

pprint(type(neos[0]['id']))
pprint(type(neos[0]['estimated_diameter']['meters']['estimated_diameter_min']))
print(type(neos))


117 Near Earth Objects found for week of 2023-01-15
{'absolute_magnitude_h': 24.86,
 'close_approach_data': [{'close_approach_date': '1949-07-04',
                          'close_approach_date_full': '1949-Jul-04 22:19',
                          'epoch_date_close_approach': -646710060000,
                          'miss_distance': {'astronomical': '0.419726307',
                                            'kilometers': '62790161.51016609',
                                            'lunar': '163.273533423',
                                            'miles': '39015997.166588442'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '73509.6739172087',
                                                'kilometers_per_second': '20.4193538659',
                                                'miles_per_hour': '45676.0526626122'}},
                         {'close_approach_date': '1950-08-07',
                        

## Exercise 1:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables:”

    id
    name
    estimated_diameter (m)
    is_potentially_hazardous (bool)

Operations must include:
* `__init__(self, ...)` method to initialize a new Asteroid object with specific data values
* `__str__(self)`, and `__repr__(self)` methods that return nicely formatted string representations of the object.

Write a little code to test your new class.

In [118]:
# Ex.1 your code here
class Asteroid:
  """
  Represents an asteroid with specific characteristics

        Attributes:
          id (int): A unique identifier for the asteroid.
          name (str): The name of the asteroid.
          estimated_diameter (float): The estimated diameter of the asteroid in metres.
          is_potentially_hazardous (bool): Whether the asteroid is potentially hazardous.
    """

  def __init__(self, id, name, estimated_diameter, is_potentially_hazardous):
    self.id = id
    self.name = name
    self.estimated_diameter =  estimated_diameter
    self.is_potentially_hazardous =  is_potentially_hazardous

  def __str__(self):
    return (f"Asteroid {self.name} (id: {self.id}) has {self.estimated_diameter} metres in diamater."
    f"{' And it IS potentially hazardous' if self.is_potentially_hazardous else ' And it is NOT potentially hazardous'}")

  def __repr__(self):
    return f"('id'={self.id},'name'={self.name}, 'estimated_diameter'={self.estimated_diameter}, 'is_potentially_hazardous'={self.is_potentially_hazardous})"


asteroid = Asteroid(12,'test',55.6,False)
print(asteroid)



Asteroid test (id: 12) has 55.6 metres in diamater. And it is NOT potentially hazardous


## Exercise 2: Make a histogram

Create a list of all the Asteroid objects from this week's NEO feed and use the list to create a histogram to visualize the distribution of Asteroid diameters.  To create the historgram use the `hist()` method in the `matplotlib.pyplot` module.

In [ ]:
# Ex. 2 your code here
date = datetime.date.today()
this_week_asteroids = get_neos(date)

average_diameter = [((asteroid['estimated_diameter']['meters']['estimated_diameter_min'] + asteroid['estimated_diameter']['meters']['estimated_diameter_max'])/2) for asteroid in this_week_asteroids]

plot = plt.hist(average_diameter, bins = 25)

plt.title(" Average asteroid diameter observed last week")
plt.xlabel("Diameter (metres)")
plt.ylabel("# of asteroids")
plt.show()



## Exercise 3: Use `@dataclass` and `@classmethod` to do the same thing

Now that you know how to define a simple class, we are going to use a use decorators to simplify (ie. shorten) our class definitions.  You will redefine the Asteroid class in this exercise.  Note that having two different definitions of a class in a single piece of code is very bad practice. I am asking you to do it here just so that you can see the differt ways to define classes.

 1. Use the `@dataclass` decorator from the `dataclasses` module to define the Asteroid class.  The `@dataclass` decorator will automatically generate some of the commonly used methods, like `__init()__`, `__repr()__`, and `__eq()__`.  The definition should start like this:
    ```
    @dataclass
    class Asteroid:
      ...
    ```
Write a little code to test your new class.
  

 2. Use the `@classmethod` decorator to construct a `from_NEO()` 'factory method' inside your Asteroid class that creates Asteroid objects from the NEO API.  The method should take the id for a single NEO, fetche the NEO record from API, constructs a single Asteroid object, and return the Asteroid object representing that NEO.  This kind of method is called a “Factory” because it constructs an object from raw materials.  The object factory method should start like this:
    ```
    @classmethod
    from_NEO(cls, neo_id):
        ...
    ```
Write a little code to test your new method.

 3. Use your new Asteroid class to create a list of Asteroid objects.  Use this list to create the same historgram as in Exercise 2.  




In [32]:
# Ex.3 your code here
@dataclass
class Asteroid:
  id: str
  name: str
  estimated_diameter: float
  is_potentially_hazardous: bool

  @classmethod
  def from_NEO(cls, neo_id: str):
    """ Fetch a NEO record for the given id and create an Asteroid object """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    asteroid_data = json.loads(response.text)


    id = asteroid_data['id']
    name = asteroid_data['name']
    estimated_diameter = ((asteroid_data['estimated_diameter']['meters']['estimated_diameter_min'] + asteroid_data['estimated_diameter']['meters']['estimated_diameter_max'])/2)
    is_potentially_hazardous = asteroid_data['is_potentially_hazardous_asteroid']

    return cls(id= id, name= name, estimated_diameter = estimated_diameter, is_potentially_hazardous= is_potentially_hazardous)


#List of Asteroid Objects
asteroid_list = [Asteroid.from_NEO(asteroid['id']) for asteroid in get_neos(date)]
pprint(asteroid_list)


84 Near Earth Objects found for week of 2025-01-24
[Asteroid(id='3709715',
          name='(2015 CL)',
          estimated_diameter=35.77195014895,
          is_potentially_hazardous=False),
 Asteroid(id='3799425',
          name='(2018 CR3)',
          estimated_diameter=32.1767826456,
          is_potentially_hazardous=False),
 Asteroid(id='3837925',
          name='(2019 BL)',
          estimated_diameter=39.22316337185,
          is_potentially_hazardous=False),
 Asteroid(id='3989084',
          name='(2020 BB5)',
          estimated_diameter=4.93790370995,
          is_potentially_hazardous=False),
 Asteroid(id='54016687',
          name='(2020 HL4)',
          estimated_diameter=51.7061994224,
          is_potentially_hazardous=False),
 Asteroid(id='54244790',
          name='(2022 CN1)',
          estimated_diameter=234.17612584425,
          is_potentially_hazardous=True),
 Asteroid(id='54414145',
          name='(2023 XZ2)',
          estimated_diameter=12.5760025529,
        

## Challenge Exercise:

One of your esteemed astromoner collegues has a theory that there is a correlation between the diameter and the orbital period of an asteroid.  Add the `orbital_period` data from the NEO API to your Asteroid class definition and use it to plot the orbital period (in years) versus the estimated diameter (in m) of all the asteroids from this week's NEO record.  Do you see a correlation?

In [ ]:
# Ex. Challenge your code here